In [41]:
#MDS Homework 4 Jay Jun 

In [42]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from math import sqrt
from random import randint

# 1. Import the data as a pd.DataFrame object. Use one or more functions from the pandas library to split the data into three unique datasets. The first should include columns that are factors only (i.e., categorical data), the second should include columns that are numeric only, and the third should include columns with logical/boolean variables only.

In [44]:
#Importing the housing price dataset. 
housing_price = pd.read_csv(r"C:\Users\jayxp\OneDrive\Desktop\MDS\Housing_prices_data.csv")

housing_factors = housing_price.select_dtypes(include = ['category', 'object'])
housing_numeric = housing_price.select_dtypes(include = ['int64', 'float64'])
housing_logic = housing_price.select_dtypes(include = ['bool'])

# 2. Using the second dataset from question #1, for each numeric variable use the apply function from the pandas library to return the mean, median and standard deviation (SD) of all numeric variables

In [46]:
def basic_stats(col):
    return pd.Series({
        'mean': col.mean(),
        'median': col.median(),
        'std': col.std()
    })

housing_numeric.apply(basic_stats, axis = 0)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
median,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883


# 3. Using the second dataset from question #1, create a new dataset that only includes the variable indicating sales price ("SalePrice"). Search for a categorical variable which has between 2-4 categories in the full dataset that you think may be related to the sales price of houses. Give the mean, median and SD for SalePrice based on the different groups of the categorical variables. Does the average price seem to vary by the different levels of the categorical factor you chose?

In [80]:
#New dataset that shows 'SalePrice' 
sale_price_numeric= housing_numeric[['SalePrice']]
# Display the first 5 rows
first_five = sale_price_numeric.head(5)

# Display the last 5 rows
last_five = sale_price_numeric.tail(5)

# Print the results
print("First Five Rows:")
print(first_five)
print("\nLast Five Rows:")
print(last_five)

First Five Rows:
   SalePrice
0     208500
1     181500
2     223500
3     140000
4     250000

Last Five Rows:
      SalePrice
1455     175000
1456     210000
1457     266500
1458     142125
1459     147500


In [84]:
#One categorical variable that has between 2-4 categories in the full dataset is 'Lotshape'. Depending on the type of lot, I think this will have a significant impact on the price.

house_style_stats = housing_price.groupby('LotShape')['SalePrice'].apply(basic_stats)
house_style_stats.head(20)

LotShape        
IR1       mean      206101.665289
          median    189000.000000
          std        85858.489559
IR2       mean      239833.365854
          median    221000.000000
          std        99669.427362
IR3       mean      216036.500000
          median    203570.000000
          std        82540.334855
Reg       mean      164754.818378
          median    146000.000000
          std        69673.427215
Name: SalePrice, dtype: float64

Yes, depending on the irrgualrity of the shape of the lot, the average sales price seems to shift. IR2 has the highest average (239833.37), then IR3 has the second highest average (216036.5), then IR1 has the third highest average (206101.67), and a regular lot has the lowest price average (164754.82). Essentially the key takeaways are that irrgular lots appear to cost more than regular lots (on average), and the differences in mean, median and standard deviation suggests that the shape of the lot appear to affect sales price.

# 4. Regress "SalePrice" on "LotArea", "OverallQual", "OverallCond" with the statsmodels library. Use the syntax below to replicate the output shown you are able to estimate this regression model.

In [52]:
# Select the dependent variable (SalePrice) and independent variables (LotArea, OverallQual, OverallCond)
X = housing_price[['LotArea', 'OverallQual', 'OverallCond']] 
y = housing_price['SalePrice']  
X = sm.add_constant(X)

# Fit the OLS regression model
model = sm.OLS(y, X).fit()

# Display the model summary
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     935.9
Date:                Sun, 23 Mar 2025   Prob (F-statistic):               0.00
Time:                        22:30:42   Log-Likelihood:                -17760.
No. Observations:                1460   AIC:                         3.553e+04
Df Residuals:                    1456   BIC:                         3.555e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.022e+05   8633.073    -11.832      

# 5. Write a for loop that loops through each level of the categorical factor you chose in Q3, where each iteration of the for loop subsets your data for one group of the categorical variable at a time, and then fits the same regression model specified above in Q4 k times, where k refers to the number of unique categories. On each iteration of your for loop, store the model.summary() information as a new element in a list object. Once your for loop is done running, print the list. There should be k regressions which you estimated. Does the effect of LotArea on SalePrice vary within the different levels of the categorical factor you chose? What is another, more common way to answer a question like this (i.e., a question about moderation or effect heterogeneity)? Hint: Don't forget to account for the standard error when comparing the regression slopes!

In [54]:
#Now I will get the each of the categories, dropping any na values (just in case), and just looking for one of each category.
categories = housing_price['LotShape'].dropna().unique()

#Now I will create an empty list to store the summaries of the regression
regression_summaries = [] 

#Creating a for loop on each level of the categorical variable 'LotShape' 
for category in categories:
    #Subsetting the data for the current category
    subset_data = housing_price[housing_price['LotShape'] == category]
    
    # Select the dependent variable (SalePrice) and independent variables (LotArea, OverallQual, OverallCond)
    X = subset_data[['LotArea', 'OverallQual', 'OverallCond']] 
    y = subset_data['SalePrice']  
    X = sm.add_constant(X)

    # Fit the OLS regression model
    model = sm.OLS(y, X).fit()

    regression_summaries.append(model.summary())

print(regression_summaries)
    

[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     676.7
Date:                Sun, 23 Mar 2025   Prob (F-statistic):          2.63e-232
Time:                        22:30:42   Log-Likelihood:                -11089.
No. Observations:                 925   AIC:                         2.219e+04
Df Residuals:                     921   BIC:                         2.220e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
c

C:\Users\jayxp\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


Yes, 'LotArea' and 'SalePrice' seems to be vary across the different regression. This suggests that the categorical factor that is used to split the data may be moderating the relationship between 'LotArea'and 'SalePrice'. The coeffecients for the different regressions were: 

Model 1 (n = 925) : 4.1066 (SE = 0.334) 95% CI (3.451, 4.762)
Model 2 (n = 484): 1.5111 (SE = 0.257) 95% CI (1.005, 2.017)
Model 3 (n = 41): 0.9666 (SE = 0.301) 95% CI (0.356, 1.577)
Model 4 (n = 10): 0.7979 (SE = 0.409) 95% CI (-0.204, 1.800)

When taking into account the standard errors as well, we can clearly see that the confidence intervals overlap, which suggests that the differences in coeffecients of 'LotArea' produced by the model may not be statistically signifgicant. The fourth model (having the smallest model, also has a very wide intervail which also includes a negative value which may indicate high uncertainity. Also from model to model, the coeffecients decrease as well suggesting some sort of moderational effect, which means that the relationship bewteen 'LotArea' and 'SalePrice' changes depending on what categorical variable was used to split the dataset. A more common way to answer a question like this is a test for moderation. 

# 6. Using the first dataset from question #1, for each categorical variable use a for loop to print a frequency and relative frequency table for every variable. To make your output readable, for each variable you should print the following in this order, placing the variable name where you see blanks below.

In [57]:
for category in housing_factors.columns:
    frequency_table = housing_price[category].value_counts(dropna=True)
    relative_frequency_table = housing_price[category].value_counts(normalize=True, dropna=True)
    
    # Here I am setting the name attribute to the category name
    frequency_table.name = category
    relative_frequency_table.name = category

    print(f"Frequency table for {category} ===============")
    print(frequency_table)

    print(f"\nRelative frequency table for {category}")
    print(relative_frequency_table)
    
    print("\n\n")





Frequency table for MSZoning ===============
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

Relative frequency table for MSZoning
MSZoning
RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64



Frequency table for Street ===============
Street
Pave    1454
Grvl       6
Name: Street, dtype: int64

Relative frequency table for Street
Street
Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64



Frequency table for Alley ===============
Alley
Grvl    50
Pave    41
Name: Alley, dtype: int64

Relative frequency table for Alley
Alley
Grvl    0.549451
Pave    0.450549
Name: Alley, dtype: float64



Frequency table for LotShape ===============
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

Relative frequency table for LotShape
LotShape
Reg    0.633562
IR1    0.331507
IR2    0.028082
IR3    0.006849
Na

# 7. Do the same thing as number 6, but with a while loop this time.he counter
    i += 1

In [59]:
# Here I am creating some sort of counter
i = 0
columns = housing_factors.columns

# While loop version
while i < len(columns):
    category = columns[i]
    frequency_table = housing_price[category].value_counts(dropna=True)
    relative_frequency_table = housing_price[category].value_counts(normalize=True, dropna=True)
    
    # Set the name attribute to the category name
    frequency_table.name = category
    relative_frequency_table.name = category

    print(f"Frequency table for {category} ===============")
    print(frequency_table)

    print(f"\nRelative frequency table for {category} ===============")
    print(relative_frequency_table)
    
    print("\n\n")
    
    # Update the counter everytime it runs
    i += 1

Frequency table for MSZoning ===============
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

Relative frequency table for MSZoning ===============
MSZoning
RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64



Frequency table for Street ===============
Street
Pave    1454
Grvl       6
Name: Street, dtype: int64

Relative frequency table for Street ===============
Street
Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64



Frequency table for Alley ===============
Alley
Grvl    50
Pave    41
Name: Alley, dtype: int64

Relative frequency table for Alley ===============
Alley
Grvl    0.549451
Pave    0.450549
Name: Alley, dtype: float64



Frequency table for LotShape ===============
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

Relative frequency table for LotShape ===============
LotShape
Re

# 8. Using the Rocket class which we created as part of the week 7 class activity, we are going to create a new class called shuttle. The shuttle class should inherit all of Rocket's characteristics but has one additional parameter: flights_completed which measures the number of flights the shuttle has completed.

Define the shuttle class as the child of the rocket class (please define it in the same jupyter notebook which has all of your other HW answers), and then initialize a new instance of your shuttle class. Once you've created a Shuttle, print the following message: "The shuttle is located at (__, __), and has recorded __ flights", filling the blank spaces to reflect the information for the Shuttle you just created.

In [61]:
class Shuttle():
    """Rocket simulates a rocket ship for a game or a physics simulation."""
    
    def __init__(self, x=0, y=0, flights_completed = 0):
        """Each rocket has an (x,y) position in 2D space. Also adding flights_complted for the third input"""
        self.x = x
        self.y = y
        self.flights_completed = flights_completed
        
    def move_rocket(self, x_increment=0, y_increment=1):
        """Moves the rocket according to the paremeters given.
           Default behavior is to move the rocket up one unit."""
        self.x += x_increment
        self.y += y_increment
        
    def get_distance(self, other_rocket):
        """Calculates the distance from this rocket to another rocket,
           and returns that value"""
        distance = sqrt((self.x-other_rocket.x)**2+(self.y-other_rocket.y)**2)
        return distance

    def flights_completed(self):
        """Counts the number of completed flights""" 
        self.flights_completed += 1

    def get_info(self):
        """Returns a string with shuttle details."""
        return f"The shuttle is located at ({self.x}, {self.y}), and has recorded {self.flights_completed} flights."

    def __str__(self):
        return self.get_info()
shuttle = Shuttle(25, 35, 12)
print(shuttle)

The shuttle is located at (25, 35), and has recorded 12 flights.


# 9 Write a single for loop which randomly creates 11 rockets and 11 shuttles using randomly generated integers. Store this output in two separate lists. Once you have your data, use for loops to print the messages you see below. Note you can use the randint() function from the random module (which you can import with the command from random import randint) to randomly generate the characteristics of your Rockets and Shuttles.

In [63]:
class Rocket():
    """Rocket simulates a rocket ship for a game or a physics simulation."""
    
    def __init__(self, x=0, y=0):
        """Each rocket has an (x,y) position in 2D space."""
        self.x = x
        self.y = y
        
    def move_rocket(self, x_increment=0, y_increment=1):
        """Moves the rocket according to the paremeters given.
           Default behavior is to move the rocket up one unit."""
        self.x += x_increment
        self.y += y_increment
        
    def get_distance(self, other_rocket):
        """Calculates the distance from this rocket to another rocket,
           and returns that value"""
        distance = sqrt((self.x-other_rocket.x)**2+(self.y-other_rocket.y)**2)
        return distance


#Let's first create a list
rockets = []
shuttles = []

for _ in range(11):  ##I have been told that _ is used for when I don't need a number. I'm looping but don't need an index.
    shuttle_x = randint(0, 100)
    shuttle_y = randint(0, 100)
    flights = randint(0,100)

    rocket_x = randint(0, 100)
    rocket_y = randint(0,100)

    rockets.append(Rocket(rocket_x, rocket_y))
    shuttles.append(Shuttle(shuttle_x, shuttle_y, flights))

#For shuttles completed flight
for i, shuttle in enumerate(shuttles):
    print(f"Shuttle {i} has completed {shuttle.flights_completed} flights.")


Shuttle 0 has completed 62 flights.
Shuttle 1 has completed 19 flights.
Shuttle 2 has completed 76 flights.
Shuttle 3 has completed 27 flights.
Shuttle 4 has completed 99 flights.
Shuttle 5 has completed 64 flights.
Shuttle 6 has completed 38 flights.
Shuttle 7 has completed 7 flights.
Shuttle 8 has completed 90 flights.
Shuttle 9 has completed 70 flights.
Shuttle 10 has completed 42 flights.


In [64]:
#Distance from a shuttle
first_shuttle = shuttles[0]
for i, shuttle in enumerate(shuttles):
    distance = first_shuttle.get_distance(shuttle)
    print(f"The first shuttle is {distance:.2f} units away from shuttle {i}.")

The first shuttle is 0.00 units away from shuttle 0.
The first shuttle is 34.18 units away from shuttle 1.
The first shuttle is 61.85 units away from shuttle 2.
The first shuttle is 33.24 units away from shuttle 3.
The first shuttle is 53.23 units away from shuttle 4.
The first shuttle is 49.24 units away from shuttle 5.
The first shuttle is 72.45 units away from shuttle 6.
The first shuttle is 19.92 units away from shuttle 7.
The first shuttle is 30.08 units away from shuttle 8.
The first shuttle is 45.54 units away from shuttle 9.
The first shuttle is 21.59 units away from shuttle 10.


In [65]:
#Distance from shuttle to rocket
for i, rocket in enumerate(rockets):
    distance = sqrt((first_shuttle.x - rocket.x)**2 + (first_shuttle.y - rocket.y)**2)
    print(f"The first shuttle is {distance:.2f} units away from rocket {i}.")

The first shuttle is 29.07 units away from rocket 0.
The first shuttle is 64.35 units away from rocket 1.
The first shuttle is 48.08 units away from rocket 2.
The first shuttle is 45.54 units away from rocket 3.
The first shuttle is 31.62 units away from rocket 4.
The first shuttle is 48.51 units away from rocket 5.
The first shuttle is 86.09 units away from rocket 6.
The first shuttle is 49.50 units away from rocket 7.
The first shuttle is 47.01 units away from rocket 8.
The first shuttle is 56.73 units away from rocket 9.
The first shuttle is 53.46 units away from rocket 10.
